In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from django.template.defaultfilters import slugify

In [55]:
root_path = os.path.dirname(os.getcwd())
root_path

'/Users/raph/code/R-a-p-h-a-e-l/Projet/ArtRecognition'

In [56]:
link = os.path.join(root_path,'ArtRecognition', 'data','all_data_info.csv')
link

'/Users/raph/code/R-a-p-h-a-e-l/Projet/ArtRecognition/ArtRecognition/data/all_data_info.csv'

In [214]:
#dff = pd.read_csv(link)
#df = dff.copy()
#df.head()

def get_df(link):
    return pd.read_csv(link)
    
df = get_df(link)
df.head()

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,Barnett Newman,1955.0,abstract,15530.0,6911.0,9201912.0,wikiart,Color Field Painting,Uriel,train_only,True,102257.jpg
1,Barnett Newman,1950.0,abstract,14559.0,6866.0,8867532.0,wikiart,Color Field Painting,Vir Heroicus Sublimis,train_only,True,75232.jpg
2,kiri nichol,2013.0,NaN,9003.0,9004.0,1756681.0,NaN,Neoplasticism,NaN,test_only,False,32145.jpg
3,kiri nichol,2013.0,NaN,9003.0,9004.0,1942046.0,NaN,Neoplasticism,NaN,test_only,False,20304.jpg
4,kiri nichol,2013.0,NaN,9003.0,9004.0,1526212.0,NaN,Neoplasticism,NaN,test_only,False,836.jpg


In [295]:
def formatting_google(txt):
    txt = str(txt)
    txt = txt.lstrip()
    txt = txt.rstrip()
    txt = txt.replace(' ',"+")
    return txt

def formatting_data(df):
    
    df["artist_format"] = df['artist'].str.lower()
    df["artist_format"] = df.artist_format.apply(slugify)
    
    df["title_format"] = df['title'].str.lower()
    
    # Year
    df["date"] = df['date'].str[:4]
    
    df['url'] = "https://www.wikiart.org/fr/"
    df["g_url"] = df['artist'].str.lower()
    
    if df["title_format"].empty==False:
        df["title_format"] = df.title_format.apply(slugify)
        df["title_format"] = df.title_format + ' ' + df["date"]
        df["title_format"] = df.title_format.apply(slugify)
        
        df['url'] = df['url'] + df['artist_format'] + "/" + df["title_format"]

        df["g_url"] = df['g_url'] + ' ' + df['title']
    
    df["g_url"] = df.g_url.apply(formatting_google)
    
    return df

In [296]:
df = formatting_data(df)
df.head()

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename,artist_format,title_format,url,g_url
0,Barnett Newman,1955,abstract,15530.0,6911.0,9201912.0,wikiart,Color Field Painting,Uriel,train_only,True,102257.jpg,barnett-newman,uriel-1955,https://www.wikiart.org/fr/barnett-newman/urie...,barnett+newman+Uriel
1,Barnett Newman,1950,abstract,14559.0,6866.0,8867532.0,wikiart,Color Field Painting,Vir Heroicus Sublimis,train_only,True,75232.jpg,barnett-newman,vir-heroicus-sublimis-1950,https://www.wikiart.org/fr/barnett-newman/vir-...,barnett+newman+Vir+Heroicus+Sublimis
2,kiri nichol,2013,NaN,9003.0,9004.0,1756681.0,NaN,Neoplasticism,NaN,test_only,False,32145.jpg,kiri-nichol,nan-2013,https://www.wikiart.org/fr/kiri-nichol/nan-2013,nan
3,kiri nichol,2013,NaN,9003.0,9004.0,1942046.0,NaN,Neoplasticism,NaN,test_only,False,20304.jpg,kiri-nichol,nan-2013,https://www.wikiart.org/fr/kiri-nichol/nan-2013,nan
4,kiri nichol,2013,NaN,9003.0,9004.0,1526212.0,NaN,Neoplasticism,NaN,test_only,False,836.jpg,kiri-nichol,nan-2013,https://www.wikiart.org/fr/kiri-nichol/nan-2013,nan


In [ ]:
#df['date_format'] = "-" + df["date"]

In [69]:
#df['source'].nunique

In [42]:
#df['url'] = "https://www.wikiart.org/fr/" + df['artist_format'] + "/" + df["title_format"]


In [70]:
#df['url'].tail(30)

In [285]:
def get_infos_by_filename(df, name, debug=False):
    ''' Return a dictionnary with informations about image '''
    new_df = df.query(f"new_filename == '{name}' ")
    if debug :
        print("-- MODE DEBUG --")
        print(new_df)
    
    new_df = new_df.to_dict('records')
    return new_df
    
informations = get_infos_by_filename(df, '20304.jpg', debug=False)
#informations[0]['g_url']
informations

[{'artist': 'kiri nichol',
  'date': '2013',
  'genre': nan,
  'pixelsx': 9003.0,
  'pixelsy': 9004.0,
  'size_bytes': 1942046.0,
  'source': nan,
  'style': 'Neoplasticism',
  'title': nan,
  'artist_group': 'test_only',
  'in_train': False,
  'new_filename': '20304.jpg',
  'artist_format': 'kiri-nichol',
  'title_format': nan,
  'url': 'https://www.wikiart.org/fr/',
  'g_url': 'kiri+nichol'}]

In [286]:
def get_infos_with_url_image(df, name, debug=False):
    # Chemin vers l'image sur GCP
    url = f"https://00f74ba44b914788edaacb1c5c6adb6fa6aeb5e777-apidata.googleusercontent.com/download/storage/v1/b/art-recognition-app/o/Clean_data%2FAll_artists%2FTrain%2FArtists_sorted_by_number_paintings%2F_1000%2F{name}"
    
    infos = get_infos_by_filename(df, name, debug=debug)
    
    # https://www.google.fr/search?q=Barnette+Newman+Uriel+1955
    search_words = infos[0]['g_url']
    g_url = "https://www.google.fr/search?q="+search_words
    
    wikiart_url = infos[0]['url']
    
    res = {
        'filename': infos[0]['new_filename'],
        'artist': infos[0]['artist'],
        'title': infos[0]['title'],
        'date': infos[0]['date'],
        'url_image': url, 
        'url_wikiart': wikiart_url, 
        'url_google': g_url}
     
    return res


In [287]:
infos = get_infos_with_url_image(df, '20304.jpg')
infos

{'filename': '20304.jpg',
 'artist': 'kiri nichol',
 'title': nan,
 'date': '2013',
 'url_image': 'https://00f74ba44b914788edaacb1c5c6adb6fa6aeb5e777-apidata.googleusercontent.com/download/storage/v1/b/art-recognition-app/o/Clean_data%2FAll_artists%2FTrain%2FArtists_sorted_by_number_paintings%2F_1000%2F20304.jpg',
 'url_wikiart': 'https://www.wikiart.org/fr/',
 'url_google': 'https://www.google.fr/search?q=kiri+nichol'}

In [293]:
def get_infos_from_image(filename):
    DEBUG = False
    df = get_df(link)
    df = formatting_data(df)
    
    infos = get_infos_with_url_image(df, filename, debug=DEBUG)
    
    return infos

In [288]:
test = get_infos_from_image('20304.jpg')

-- MODE DEBUG --
        artist  date genre  pixelsx  pixelsy  size_bytes source  \
3  kiri nichol  2013   NaN   9003.0   9004.0   1942046.0    NaN   

           style title artist_group  in_train new_filename artist_format  \
3  Neoplasticism   NaN    test_only     False    20304.jpg   kiri-nichol   

  title_format                          url        g_url  
3          NaN  https://www.wikiart.org/fr/  kiri+nichol  


In [302]:
def get_image_directory(filename):
    
    path = os.path.join(root_path,'ArtRecognition', 'data','artists_numbers.csv')
    class_df = pd.read_csv(path)
    
    infos = get_infos_from_image(filename)
    artist = infos['artist']
    
    artist = 'John Singer Sargent'
    cdf = class_df.query(f"artist == '{artist}' ")
    classes = cdf.to_dict('records')
    directory = classes['artist_number']

    return directory

df2 = get_image_directory('20304.jpg')
df2

TypeError: list indices must be integers or slices, not str